# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846841818608                   -0.70    4.5
  2   -7.852307996201       -2.26       -1.53    1.0
  3   -7.852615645775       -3.51       -2.56    1.5
  4   -7.852645956332       -4.52       -2.89    2.5
  5   -7.852646511306       -6.26       -3.20    1.2
  6   -7.852646679268       -6.77       -4.07    1.0
  7   -7.852646686691       -8.13       -5.28    2.2
  8   -7.852646686725      -10.46       -5.48    1.8
  9   -7.852646686727      -11.73       -5.56    1.2
 10   -7.852646686730      -11.59       -6.75    1.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846775178548                   -0.70           4.5
  2   -7.852551901121       -2.24       -1.63   0.80    2.2
  3   -7.852640000360       -4.06       -2.71   0.80    1.0
  4   -7.852646420166       -5.19       -3.40   0.80    1.8
  5   -7.852646682312       -6.58       -4.24   0.80    2.2
  6   -7.852646686614       -8.37       -4.83   0.80    1.8
  7   -7.852646686718       -9.98       -5.80   0.80    1.2
  8   -7.852646686730      -10.94       -6.28   0.80    2.5


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.352738e+01     3.708230e+00
 * time: 0.43405818939208984
     1     1.610146e+00     1.927271e+00
 * time: 0.6559610366821289
     2    -1.547074e+00     2.314557e+00
 * time: 0.6835250854492188
     3    -3.603128e+00     1.771096e+00
 * time: 0.724567174911499
     4    -5.009949e+00     1.461763e+00
 * time: 0.7645220756530762
     5    -6.631694e+00     1.082323e+00
 * time: 0.803901195526123
     6    -7.311454e+00     6.447777e-01
 * time: 0.8435850143432617
     7    -7.590539e+00     5.265649e-01
 * time: 0.8713021278381348
     8    -7.720535e+00     8.002090e-02
 * time: 0.8988971710205078
     9    -7.758775e+00     1.817943e-01
 * time: 0.9267120361328125
    10    -7.785797e+00     9.025668e-02
 * time: 0.9540691375732422
    11    -7.802759e+00     9.845955e-02
 * time: 0.9816880226135254
    12    -7.822301e+00     6.953764e-02
 * time: 1.009505033493042
    13    -7.834289e+00     5.570577e-02
 * time: 1.037266016006

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846682371935                   -0.70    4.2


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852645789188                   -1.64
  2   -7.852646686730       -6.05       -3.70
  3   -7.852646686730      -13.04       -7.15


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.595961232655167e-7
|ρ_newton - ρ_scfv| = 2.12595950822635e-7
|ρ_newton - ρ_dm|   = 2.4720602321336543e-9
